# Configure Batch AI
Create resources needed for Batch AI, and create the configuration file used by Batch AI utilities.

The steps are
- [import the libraries](#import),
- [add to the dotenv shared parameters](#dotenv),
- [login to the Azure account](#login),
- [create resources needed to use Batch AI](#resources), and
- [write the Batch AI configuration file](#configuration).


## Imports <a id='import'></a>

In [ ]:
%load_ext dotenv
import os
import shutil
import json
import dotenv

## Set up  <a id='dotenv'></a>
Define parameters used to create resources for Batch AI. Please modify the values below as you see fit.

In [ ]:
# The location of the dotenv file
dotenv_path = dotenv.find_dotenv()
# If you have multiple subscriptions select the subscription you want to use
dotenv.set_key(dotenv_path, 'selected_subscription', "selected_subscription")
# To contain all resources to be created
dotenv.set_key(dotenv_path, 'resource_group', "resource_group")
# The region in which resources will be located
dotenv.set_key(dotenv_path, 'location', "location") 
# The default Batch AI workspace to be created
dotenv.set_key(dotenv_path, 'batchai_workspace', "batchai_workspace")
# The Batch AI administrator account login
dotenv.set_key(dotenv_path, 'batchai_login', "batchai_login")
# The Batch AI administrator account password
dotenv.set_key(dotenv_path, 'batchai_password', "batchai_password")
# Storage for the data, learning script, and outputs
dotenv.set_key(dotenv_path, 'storage_account', "storage_account")
# The file containing the Batch AI configuration
dotenv.set_key(dotenv_path, 'configuration_path', 'configuration_path')

Read in the parameters.

In [ ]:
%dotenv -o

## Azure account login <a id='login'></a>
The commands below will ensure you are logged in to your Azure account. If you are not logged in, it will initiate a login by popping up a browser at a login page where you will select your Azure account.

In [ ]:
%%bash
list=`az account list -o table`
if [ "$list" == '[]' ] || [ "$list" == '' ]; then 
  az login -o table
else
  az account list -o table 
fi

Set the selected subscription as the default.

In [ ]:
%%bash
az account set --subscription "$selected_subscription"
az account show -o table

Get the information for the selected Azure subscription.

In [ ]:
account_json = !az account show
account = json.loads(''.join(account_json))

## Batch AI resources <a id='resources'></a>
Register the Batch and Batch AI resources for your subscription. This may take a few seconds to become effective.

In [ ]:
%%bash
az provider register -n Microsoft.Batch
az provider register -n Microsoft.BatchAI

Assign to Batch AI the role of Active Directory Network Contributor so it can manage clusters for you. The assignee ```9fcb3732-5f52-4135-8c08-9d4bbaf203ea``` is the service principal of Microsoft Azure BatchAI in public cloud.

In [ ]:
!az role assignment create --role "Network Contributor" --assignee 9fcb3732-5f52-4135-8c08-9d4bbaf203ea

Create the resource group if it doesn't already exist. It will hold the Batch AI workspace and a storage accounnt. 

In [ ]:
!az group create -l $location -n $resource_group -o table

Create the storage account. If it doesn't already exist, this step may take many minutes.

In [ ]:
!az storage account create -l $location -g $resource_group -n $storage_account -o table

Get a key to the storage account.

In [ ]:
storage_keys_json = !az storage account keys list -g $resource_group -n $storage_account
storage_keys = json.loads(''.join(storage_keys_json))

Create a service principal for Batch AI and configure its access to Azure. Capture and parse the output for later use.

In [ ]:
aad_json = !az ad sp create-for-rbac
aad = json.loads(''.join(aad_json[aad_json.index('{'):]))

Create the workspace if it doesn't already exist.

In [ ]:
!az batchai workspace create -l $location -g $resource_group -n $batchai_workspace -o table

## Configuration dictionary <a id='configuration'></a>
Use the above information to create a configuration dictionary needed by various Batch AI utilities.

In [ ]:
cfg = {
    "subscription_id": account['id'],
    "aad_client_id": aad['appId'],
    "aad_secret": aad['password'],
    "aad_tenant": aad['tenant'],
    "location": os.getenv('location'),
    "base_url": "",
    "resource_group": os.getenv('resource_group'),
    "workspace": os.getenv('batchai_workspace'),
    "storage_account" : {
        "name": os.getenv('storage_account'),
        "key": storage_keys[0]['value']
    },
    "admin_user" : {
        "name": os.getenv('batchai_login'),
        "password": os.getenv('batchai_password'),
        "ssh_public_key": ""
    },
    "keyvault_id": "",
    "container_registry" : {
        "user": "",
        "password": "",
        "secret_url": ""
    }
}

Write the configuration to a JSON file.

In [ ]:
configuration_path = os.getenv('configuration_path')
with open(configuration_path, 'w') as fp:
    json.dump(cfg, fp, indent=4, sort_keys=True)

In the next [notebook](04_Create_Cluster.ipynb), we will create a cluster using the Batch AI framework.